In [78]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [79]:
rawdata = pd.read_csv("Churn_Modelling.csv")

In [80]:
rawdata

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Row Number, Customer ID, Surname won't play a role in prediction hence they are dropped. 

In [81]:
#Matrix of Features
x = rawdata.iloc[:,3:-1].values

In [82]:
#Matrix of Target variable
y = rawdata.iloc[:,-1].values

Country and Gender are categorical data

In [83]:
#Gender - Label Encoding

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:,2] = le.fit_transform(x[:,2])


In [84]:
#Country - 1-Hot Encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')

x = np.array(ct.fit_transform(x))

In [85]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2, random_state=0)

Apart from usual, Scaling 1) improves optimization, hence smoothens the gradient descent 2) helps minimize the cost function quickly.

In [86]:
#Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [87]:
#Initialize ANN Model
ann = tf.keras.Sequential()

units - randomly chosen, no right answer, comes from experience or hit & trial

In [88]:
# Add first, hidden and second layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Output Layer - Prediction of binary output (Churn - Yes or No).

In [89]:
#Add output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Model Compiling - Define optimizer, loss, metrics

In [90]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [91]:
#Train Model
ann.fit(X_train, Y_train, epochs = 100)

Epoch 1/100


250/250 [==============================] - 1s 574us/step - loss: 0.6068 - accuracy: 0.7216
Epoch 2/100
250/250 [==============================] - 0s 570us/step - loss: 0.4845 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 565us/step - loss: 0.4585 - accuracy: 0.7997
Epoch 4/100
250/250 [==============================] - 0s 575us/step - loss: 0.4457 - accuracy: 0.8048
Epoch 5/100
250/250 [==============================] - 0s 627us/step - loss: 0.4377 - accuracy: 0.8086
Epoch 6/100
250/250 [==============================] - 0s 587us/step - loss: 0.4318 - accuracy: 0.8102
Epoch 7/100
250/250 [==============================] - 0s 642us/step - loss: 0.4266 - accuracy: 0.8142
Epoch 8/100
250/250 [==============================] - 0s 592us/step - loss: 0.4208 - accuracy: 0.8166
Epoch 9/100
250/250 [==============================] - 0s 568us/step - loss: 0.4125 - accuracy: 0.8201
Epoch 10/100
250/250 [==============================] - 0s 598us/step - loss: 0.4025 

In [92]:
#Weights and bias from last hidden layer to output layer
ann.layers[2].get_weights()

[array([[-0.8392728 ],
        [-1.1998901 ],
        [ 1.2809789 ],
        [ 0.86701953],
        [-0.25240153],
        [ 0.9309343 ]], dtype=float32),
 array([-0.33032572], dtype=float32)]

Making a Prediction

In [93]:
NewCustomer = np.array([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])

In [94]:
#probability of leaving
ann.predict(sc.transform(NewCustomer))

1/1 [==============================] - 0s 45ms/step


array([[0.03020105]], dtype=float32)

In [95]:
#Test Set Results
y_pred = ann.predict(X_test)

63/63 [==============================] - 0s 515us/step


In [96]:
#convert probability to Yes or No
Y_pred = (y_pred > 0.5)

In [97]:
Y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [98]:
print(np.concatenate((Y_pred.reshape(len(Y_pred),1), Y_test.reshape(len(Y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [99]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,Y_pred)
print(cm)

[[1508   87]
 [ 192  213]]


In [100]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8605